<a href="https://colab.research.google.com/github/torontoniancoder/python_tutorial/blob/master/ML_with_linear_regression_in_Python_final_project_C05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
import pandas as pd
import numpy as np

A. Data cleaning

In [122]:
def clean_data(df):
    pd.set_option('mode.use_inf_as_na', True) # to recognise inf as a NaN in calculations: from here: https://stackoverflow.com/questions/58225701/change-pandas-default-fillna-setting
    # todo: replace the nan values with relevant (??) values (MEAN?) -> replaced with 0-es
    # selecting the columns for conversion from categorical data into columns of binary data
    df_temp = df[['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years']]
    # integer encoding
    # source for instructions: https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40
    # turning the type columns into the dummy variables
    # No nan values therefore dummy_na=False
    dummy = pd.get_dummies(df_temp, dummy_na=False, columns=['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], prefix=['Product_ID','Gender','Age','Occupation', 'City_Category','Stay_In_Current_City_Years'], dtype=int, drop_first=False)
    # concatenating binary data into the initial dataframe
    df = pd.concat([df,dummy], axis=1)
    # dropping the unnecessary fields
    # FieldDrop0 = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Purchase']
    # df = df.drop(FieldDrop0, axis=1)
    colnames = df.columns.values.tolist() #converting column names into a list
    # colnames
    # counting nan values and removing columns with nan values more than 50% (threshold can be modified *there -> <-*)
    for each in colnames:
        # print(type(df[each]))
        if (( df[each].isna().sum() / len(df[each]) ) > 0.5): # *-> <-* here! the threshold can be modified | source URL: https://stackoverflow.com/questions/26266362/how-to-count-the-nan-values-in-a-column-in-pandas-dataframe
            print ('Due to low data quality\n(data not available for more than 50% of data points)\nI am deleting this column: '+each)
            # *-> <-* here! in the line above the threshold must be modified if it si changed in the previous line
            del df[each]
        else:
            df[each] = df[each].fillna(0)
            df[each] = df[each].astype('string')
            pass
            # z = ( df[each].isna().sum() / len(df[each]) )*100
            # print('For column '+str(each)+' unknown datapoints are '+str(z)+'% of series')
    # df.info()
    # df = df.astype('string') # source URL: https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    # df[["D", "E"]] = df[["D", "E"]].astype(int)
    #   cleaning temporary data
    df_temp = 0
    return df

In [123]:
df = pd.read_csv('Black_Friday.csv') #loading data into a dataframe

In [124]:
# *** REMOVE BEFORE ASSEMBLY *** BEGIN
df = df.iloc[:1000] # slicing dataframe for test&dev performance
# df.shape
# *** REMOVE BEFORE ASSEMBLY *** END


In [125]:
# keeping a control copy of the source dataset in variable df0
df0 = df.copy()

In [126]:
df = clean_data(df) # running function clean_data on a dataframe

Due to low data quality
(data not available for more than 50% of data points)
I am deleting this column: Product_Category_3


In [127]:
df.info()
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 139 entries, User_ID to Stay_In_Current_City_Years_4+
dtypes: string(139)
memory usage: 108.7 KB


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,Product_ID_P00014542,Product_ID_P00019342,Product_ID_P00031342,Product_ID_P00032442,Product_ID_P00034442,Product_ID_P00036842,Product_ID_P00039942,Product_ID_P00042142,Product_ID_P00044142,Product_ID_P00058142,Product_ID_P00058342,Product_ID_P00058442,Product_ID_P00069042,Product_ID_P00073842,Product_ID_P00078742,Product_ID_P00085442,Product_ID_P00085942,Product_ID_P00087242,Product_ID_P00087842,Product_ID_P00100642,Product_ID_P00105942,Product_ID_P00110842,Product_ID_P00111142,Product_ID_P00112542,Product_ID_P00112642,Product_ID_P00113242,Product_ID_P00117442,Product_ID_P00118342,Product_ID_P00118442,...,Product_ID_P00334242,Product_ID_P00338442,Product_ID_P00339542,Product_ID_P00346142,Product_ID_P00348242,Product_ID_P00351142,Product_ID_P00365242,Product_ID_P00366542,Product_ID_P0094142,Product_ID_P0094542,Product_ID_P0096642,Product_ID_P0097242,Gender_F,Gender_M,Age_0-17,Age_18-25,Age_26-35,Age_36-45,Age_46-50,Age_51-55,Age_55+,Occupation_0,Occupation_1,Occupation_3,Occupation_7,Occupation_9,Occupation_10,Occupation_12,Occupation_15,Occupation_16,Occupation_17,Occupation_20,City_Category_A,City_Category_B,City_Category_C,Stay_In_Current_City_Years_0,Stay_In_Current_City_Years_1,Stay_In_Current_City_Years_2,Stay_In_Current_City_Years_3,Stay_In_Current_City_Years_4+
count,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
unique,21,100,2,7,11,3,5,2,10,16,100,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,1000010,P00250642,M,26-35,1,B,3,0,1,0.0,5958,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,18,1,54,30,28,38,31,62,33,23,1,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,...,99,99,99,99,99,99,99,99,99,99,99,99,54,54,84,82,70,78,94,93,99,97,72,85,88,96,84,92,97,98,96,95,63,62,75,89,78,91,69,73


C. Data Modeling

In [128]:
# source code URL: https://habr.com/ru/post/206306/
# and here, URL: https://habr.com/ru/post/202090/

# importing libraries

from pandas import read_csv, DataFrame
# from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.svm import SVR
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import r2_score
# from sklearn import cross_validation, svm
# from sklearn.cross_validation import train_test_split

In [129]:
# creating dataset with summary data per UserID

# df_c1 = df.copy() #loading data into a dataframe

In [130]:
# df_c1 = clean_data(df_c1) #
# PreFinal_df = df.groupby('User_ID').sum()

In [131]:
# df_c1.info()

In [132]:
# df_c1 = df_c1.astype(str)

In [133]:
# df_c1.info()

In [134]:
# To begin with, we need to divide our training sample
# into the indicator that we are examining, and its defining features:
# todo: check features for cross-correlation, see source code
#  here, URL: http://espressocode.top/python-pandas-dataframe-corr/

# PreFinal_df

In [135]:
FieldDrop_PID = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years']
df = df.drop(FieldDrop_PID, axis=1)

In [136]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Columns: 132 entries, Marital_Status to Stay_In_Current_City_Years_4+
dtypes: string(132)
memory usage: 103.2 KB


In [137]:
df = df.astype(float)
# df.shape()
# df.info()

In [138]:
CorrKoeff = df.corr()

In [139]:
# CorrKoeff

In [140]:
colnames = 0

In [141]:
colnames = CorrKoeff.columns.values.tolist()

In [142]:
Feature_Drop = list(())
Feature_DropSoft = list(())

In [143]:
# checking the correlation between the features
for each in colnames:
    i = len(CorrKoeff[each])
    for i in range(i):
        if (CorrKoeff.iloc[i][each]  > 0.95) and (CorrKoeff.index[i] != each):
            # print('\n* *XX*|||*XX* * !!WARNING!! ['+str(each)+'] IS CORRELATED WITH '+str(CorrKoeff.index[i])+' BY '+str(CorrKoeff.iloc[i][each])+' * *XX*|||*XX* *')
            Feature_Drop.append(each)
            pass
        elif (CorrKoeff.iloc[i][each]  < -0.95) and (CorrKoeff.index[i] != each):
            # print('\n- *XX*XX* - Attention: ['+str(each)+'] is correlated with ['+str(CorrKoeff.index[i])+'] by '+str(CorrKoeff.iloc[i][each])+' - *XX*XX* -\n')
            Feature_DropSoft.append(each)
            pass
        else:
            pass
    # print('----- * ----- '+str(each)+' is [COMPLETE] ----- * -----')

In [144]:
# there is a correlation > 90% and <-90% between the features in df

In [145]:
# todo: drop the unnecessary features (columns)

In [146]:
# writing the Purchase data (targeted for prediction) into a separate dataset
target = df0['Purchase']
# dropping the unnecessary features


In [147]:
# FieldDrop1 = ['Purchase']

In [148]:
# df = df.drop(FieldDrop1, axis=1)

In [149]:
# df

In [150]:
len(Feature_Drop)

4

In [151]:
len(Feature_DropSoft)

2

In [152]:
# Feature_DropSoft = Feature_DropSoft + Feature_Drop
# len(Feature_DropSoft)

In [153]:
UFeatures_Drop = list(())
UFeatures_DropSoft = list(())

In [154]:
[UFeatures_Drop.append(x) for x in Feature_Drop if x not in UFeatures_Drop]
[UFeatures_DropSoft.append(z) for z in Feature_DropSoft if z not in UFeatures_DropSoft]

[None, None]

In [155]:
length = len(UFeatures_Drop)
lengthSoft = len(UFeatures_DropSoft)

In [156]:
middle_index = length//2 # source code URL: https://www.kite.com/python/answers/how-to-split-a-list-in-half-in-python
middle_indexSoft = lengthSoft//2

In [157]:
UFeatures_Droph = UFeatures_Drop[:middle_index]
UFeatures_DropSofth = UFeatures_DropSoft[:middle_index]

In [158]:
UFeatures_Drop_United = UFeatures_Droph + UFeatures_DropSofth

In [159]:
len(UFeatures_Drop_United)

4

In [160]:
# df

In [161]:
newdf = df.drop(UFeatures_Drop_United, axis=1).copy()

In [162]:
# newdf

In [163]:
# newdf2 = newdf.drop(Feature_DropSoft, axis=1).copy()

In [164]:
# newdf2.shape()

In [165]:
# newtr1 = newtr[['Gender_F','Gender_M','Age_0-17','Age_18-25','Age_26-35','Age_36-45','Age_46-50','Age_51-55','Age_55+','City_Category_A','City_Category_B','City_Category_C','Stay_In_Current_City_Years_0','Stay_In_Current_City_Years_1','Stay_In_Current_City_Years_2','Stay_In_Current_City_Years_3','Stay_In_Current_City_Years_4+']].copy()

In [166]:
# models = [LinearRegression(), # метод наименьших квадратов
#	          RandomForestRegressor(n_estimators=100, max_features ='sqrt'), # случайный лес
#	          KNeighborsRegressor(n_neighbors=6), # метод ближайших соседей
#	          SVR(kernel='linear'), # метод опорных векторов с линейным ядром
#	          LogisticRegression() # логистическая регрессия
#	          ]



In [167]:
from sklearn.model_selection import train_test_split

In [168]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(newdf, target, test_size=0.2)


In [169]:
print (Xtrn.shape)
print (Ytrn.shape)
print (Xtest.shape)
print (Ytest.shape)
# print X_test.shape, y_test.shape
# source code URL: https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

(80, 128)
(80,)
(20, 128)
(20,)


In [170]:
# fit a model
lm = LinearRegression()
model = lm.fit(Xtrn, Ytrn)
predictions = lm.predict(Xtest)


In [171]:
# Let’s see what (some of) the predictions are:
predictions[0:5]

array([15853.99999947,  3957.00000284,  7746.00000535, 15911.99999924,
        8290.00000277])

In [172]:
Ytest = Ytest.astype(float)
predictions = predictions.astype(float)

In [173]:
# source code URL: https://python-school.ru/linear-regression-basis/

# Ytest = np.exp(Ytest)
# predictions = np.exp(predictions)

In [174]:
# source code URL: https://python-school.ru/linear-regression-basis/
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(Ytest, predictions)
mae = mean_absolute_error(Ytest, predictions)
print('mse: %.3f, mae: %.3f' % (mse, mae))

mse: 0.000, mae: 0.000


In [175]:
# import seaborn as sns

In [176]:
# sns.pairplot(newdf)


In [177]:
# features = newdf.columns.values.tolist()

In [178]:
# sns.pairplot(newdf, x_vars=[features], y_vars='predictions', kind='reg', height=100)